In [7]:
import pandas as pd
from lemmatize import lemmatize
from llms import gemini
from llms import chatGPT
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime

x_chat = chatGPT()
x_gemini = gemini()

def parse_date(date):
    if "," in date:
        return datetime.strptime(date, "%b %d, %Y")
    elif len(date)==4:
        return datetime.strptime(date, "%Y")
    else:
        return datetime.strptime(date, "%b %Y")

NotFoundError: Error code: 404 - {'error': {'message': 'The model `h` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [4]:
data = open("../input/years/2019/data2019.txt").read().split("\n\n")
data = [[y for y in x.split("\n") if y != "" and "Logo" not in y] for x in data if x != ""]
bad = [[len(x),x] for x in data if len(x) != 22]
good2019 = [x for x in data if len(x) == 22]
print(len(good2019))

data = open("../input/years/2020/data2020.txt").read().split("\n\n")
data = [[y for y in x.split("\n") if y != "" and "Logo" not in y] for x in data if x != ""]
bad = [[len(x),x] for x in data if len(x) != 22]
good2020 = [x for x in data if len(x) == 22]
print(len(good2020))

data = open("../input/years/2021/data2021.txt").read().split("\n\n")
data = [[y for y in x.split("\n") if y != "" and "Logo" not in y] for x in data if x != ""]
bad = [[len(x),x] for x in data if len(x) != 22]
good2021 = [x for x in data if len(x) == 22]
print(len(good2021))

data = open("../input/years/2022/data2022.txt").read().split("\n\n")
data = [[y for y in x.split("\n") if y != "" and "Logo" not in y] for x in data if x != ""]
bad = [[len(x),x] for x in data if len(x) != 22]
good2022 = [x for x in data if len(x) == 22]
#weird formatting when scrapping
good2022 = [x[:9]+x[10:11]+x[9:10]+x[11:] for x in data if len(x) == 22]
print(len(good2022))

data = open("../input/years/2023/data2023.txt").read().split("\n\n")
data = [[y for y in x.split("\n") if y != "" and "Logo" not in y] for x in data if x != ""]
bad = [[len(x),x] for x in data if len(x) != 22]
good2023 = [x for x in data if len(x) == 22]
print(len(good2023))

data = open("../input/years/2024/data2024.txt").read().split("\n\n")
data = [[y for y in x.split("\n") if y != "" and "Logo" not in y] for x in data if x != ""]
bad = [[len(x),x] for x in data if len(x) != 22]
good2024 = [x for x in data if len(x) == 22]
print(len(good2024))

labels = ["organization name","num employees", "founded date","description","industries","headquarters location","description","cb rank","postal code","headquarters region","website","actively hiring","linkedin","num articles","email","hub tag","phone num","num of sub org","industry groups","total funding amt","company type","estimated rev"]
dataset = good2019+ good2020 + good2021 + good2022 + good2023 + good2024
df = pd.DataFrame(dataset,columns=labels)

df['description_all'] = df['description'].iloc[:,0] + " " + df['description'].iloc[:,1]
df['industries_parsed'] = df['industries'].apply(lambda x: [i.strip() for i in x.split(',') if i.strip() != 'Artificial Intelligence (AI)' and i.strip() !='—'])
# df['founded date'] = pd.to_datetime(df['founded date'], format='%b %Y', errors='coerce').combine_first(pd.to_datetime(df['founded date'], errors='coerce', infer_datetime_format=True))

df["founded date"] = df["founded date"].apply(lambda x: parse_date(x))
df.to_csv("../output/data/data.csv")

4669
4558
4349
3802
4266
987


In [ ]:
pd.DataFrame(good2022,columns=labels)

In [39]:
df = df[df["headquarters location"].str.contains("United States")]
df = df[pd.to_datetime(df['founded date'])>'11/30/2022']
df.website

16817                 stack-ai.com
16818              trykintsugi.com
16820             www.sesameai.com
16823     procurementsciences.com/
16826               www.catio.tech
                   ...            
22624             www.supallama.ai
22625              www.geolava.com
22626                   solatis.ai
22627    www.degreesofinterest.com
22628                  aicomply.ai
Name: website, Length: 1304, dtype: object

In [3]:
codes

,U.S. Bureau of Labor Statistics,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,On behalf of the Office of Management and Budg...,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,November 2017 (for reference year January 2018),NaN,NaN,NaN,NaN
3,***This is the final structure for the 2018 SO...,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
1449,NaN,NaN,NaN,55-3014,Artillery and Missile Crew Members
1450,NaN,NaN,NaN,55-3015,Command and Control Center Specialists
1451,NaN,NaN,NaN,55-3016,Infantry
1452,NaN,NaN,NaN,55-3018,Special Forces


In [5]:
import pandas as pd
codes = pd.read_csv("../input/soc_codes/soc_structure_2018.csv")

codes['Major Group Name'] = codes.apply(lambda x: x[4] if pd.notnull(x['Major Group']) else None, axis=1)
codes['Minor Group Name'] = codes.apply(lambda x: x[4] if pd.notnull(x['Minor Group']) else None, axis=1)
codes['Broad Group Name'] = codes.apply(lambda x: x[4] if pd.notnull(x['Broad Group']) else None, axis=1)

codes['Major Group'] = codes['Major Group'].fillna(method='ffill')
codes['Minor Group'] = codes['Minor Group'].fillna(method='ffill')
codes['Broad Group'] = codes['Broad Group'].fillna(method='ffill')
codes['Major Group Name'] = codes['Major Group Name'].fillna(method='ffill')
codes['Minor Group Name'] = codes['Minor Group Name'].fillna(method='ffill')
codes['Broad Group Name'] = codes['Broad Group Name'].fillna(method='ffill')

codes = codes[codes["Detailed Occupation"].notna()]
# codes.to_csv("../input/soc_codes/all_soc_codes.csv")
codes = codes.rename({"Unnamed: 4":"Title"},axis=1)
codes.to_csv("../input/soc_codes/soc_codes.csv")


/var/folders/nf/2dvvlnxj5v962kwwzmpfj17h0000gn/T/ipykernel_18740/1325248018.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  codes['Major Group Name'] = codes.apply(lambda x: x[4] if pd.notnull(x['Major Group']) else None, axis=1)
/var/folders/nf/2dvvlnxj5v962kwwzmpfj17h0000gn/T/ipykernel_18740/1325248018.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  codes['Minor Group Name'] = codes.apply(lambda x: x[4] if pd.notnull(x['Minor Group']) else None, axis=1)
/var/folders/nf/2dvvlnxj5v962kwwzmpfj17h0000gn/T/ipykernel_18740/1325248018.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated

In [8]:
df_exp = pd.read_csv("../input/gpts_labels_new.csv")
df_exp = df_exp[(df_exp.human_labels!='E0') | (df_exp.gpt_4_exposure!='E0')]
df_exp["description"] = df_exp.apply(lambda x: f"{x.Title.capitalize()} that {x.Task.lower()}",axis=1)
df_exp[(df_exp.gpt_4_exposure=='EO')]
df_exp.gpt_4_exposure.unique()

df_exp["description_lemmatize"] = df_exp.description.apply(lambda x: lemmatize(x))
df_exp["title_lemmatize"] = df_exp.Title.apply(lambda x: lemmatize(x))
df_exp = df_exp.rename({"Title":"title"},axis=1)

In [12]:
# # results = []
# for i, x in list(df_exp.reset_index().iterrows())[8914:]:
#     # description_lemmatize = x["description_lemmatize"]
#     # title_lemmatize = x["title_lemmatize"]
#     description = x["description"]
#     title = x["title"]
#     print(f"Processing {str(i), len(results)}: {description}")
#     result_description = x_chat.get_embedding(description)
#     result_title = x_chat.get_embedding(title)
#     results.append([x["Task ID"],result_description, result_title])

# results_df = pd.DataFrame(results, columns=['Task ID', 'task_embedding','title_embedding'])
# df_final = df_exp.merge(results_df, on="Task ID")



# results_df = pd.DataFrame(results, columns=['Task ID', 'task_embedding','title_embedding'])
# df_final = df_exp.merge(results_df, on="Task ID")
# df_final.to_csv("../output/gpt_exposure_embeddings.csv")

NameError: name 'results' is not defined

In [ ]:
df_exp = pd.read_csv("../output/onet/gpt_exposure_embeddings-v1.csv")

In [9]:
batch_size = 200

# Run batch embeddings for a specific column (e.g., 'description')
embeddings_df = x_chat.run_batch_embeddings(df_exp, col='description', batch_size=batch_size)


0        Sales managers that resolve customer complaint...
1        Sales managers that monitor customer preferenc...
2        Sales managers that determine price schedules ...
3        Sales managers that review operational records...
4        Sales managers that confer or consult with dep...
                               ...                        
19246    Aircraft service attendants that complete form...
19250    Aircraft service attendants that inspect aircr...
19254    Aircraft service attendants that radio to flig...
19260    Transportation inspectors that calculate gross...
19262    Transportation inspectors that visually inspec...
Name: description, Length: 11582, dtype: object
0        Sales managers that resolve customer complaint...
1        Sales managers that monitor customer preferenc...
2        Sales managers that determine price schedules ...
3        Sales managers that review operational records...
4        Sales managers that confer or consult with dep...
        

KeyboardInterrupt: 

In [13]:
df_final.to_csv("../output/onet/gpt_exposure_embeddings-v1.csv")

In [ ]:

# Merge the embeddings with the original DataFrame
df_final = df_exp.join(embeddings_df, on=df_exp.index)

# Optionally save df_final to a file
df_final.to_csv('df_final.csv', index=False)

In [4]:
codes

,Major Group,Minor Group,Broad Group,Detailed Occupation,Title,Major Group Name,Minor Group Name,Broad Group Name
0,11-0000,NaN,NaN,NaN,Management Occupations,Management Occupations,None,None
1,11-0000,11-1000,NaN,NaN,Top Executives,Management Occupations,Top Executives,None
2,11-0000,11-1000,11-1010,NaN,Chief Executives,Management Occupations,Top Executives,Chief Executives
3,11-0000,11-1000,11-1010,11-1011,Chief Executives,Management Occupations,Top Executives,Chief Executives
4,11-0000,11-1000,11-1020,NaN,General and Operations Managers,Management Occupations,Top Executives,General and Operations Managers
...,...,...,...,...,...,...,...,...
1442,55-0000,55-3000,55-3010,55-3014,Artillery and Missile Crew Members,Military Specific Occupations,Military Enlisted Tactical Operations and Air/...,Military Enlisted Tactical Operations and Air/...
1443,55-0000,55-3000,55-3010,55-3015,Command and Control Center Specialists,Military Specific Occupations,Military Enlisted Tactical Operations and Air/...,Military Enlisted Tactical Operations and Air/...
1444,55-0000,55-3000,55-3010,55-3016,Infantry,Military Specific Occupations,Military Enlisted Tactical Operations and Air/...,Military Enlisted Tactical Operations and Air/...
1445,55-0000,55-3000,55-3010,55-3018,Special Forces,Military Specific Occupations,Military Enlisted Tactical Operations and Air/...,Military Enlisted Tactical Operations and Air/...


In [2]:
df

,h
0,j
